In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
txt = "Data/Traverse.txt"
df = pd.read_csv(txt, sep = "\t")

In [3]:
df.nunique()
df.drop(columns = ['desc', 'kml_folder', 'type'], inplace = True)

df.rename(columns={'name': 'description'}, inplace=True)
df['description'].ffill(inplace=True)


In [72]:
gdf = gpd.GeoDataFrame(data = df,
                        crs = "EPSG:4326",
                        geometry = gpd.points_from_xy(df['longitude'], df['latitude'])) # type: ignore

gdf = gdf.to_crs('EPSG:32737')                        

gdf.drop(columns = ['latitude', 'longitude'], inplace = True)
gdf_ = gdf[['altitude (m)', 'description']]

In [73]:
# Transformation

e = 1618010.606367111
f = -1741978.211608887

a = 0.9916765364496314
d = 1.175648448821448

b = -0.1635616585599564
c = 0.02551828620198648

# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
gdf2.rename({0 : 'geometry'}, axis = 1, inplace = True)

# gdf2.head()

In [74]:
gdf3 = pd.concat([gdf_, gdf2], axis = 1)
gdf3 = gpd.GeoDataFrame(gdf3, crs = "EPSG:32737")
# gdf3.head(30)

In [75]:
df_ = gdf3
df_['x'] = gdf3.geometry.x
df_['y'] = gdf3.geometry.y
df_.drop("geometry", axis = 1, inplace = True)
df_['P'] = df_.index + 1

names ={
    "description" : "D",
    "altitude (m)" : "Z",
    "x" : "E",
    "y" : "N"
    }

df_.rename(columns = names, inplace = True)
df_ = df_[['P', 'N', 'E', 'Z', 'D']]
# df_

In [76]:
# df_.iloc[1:13, 4].replace('Traverse','Tr', inplace = True)
i = 1
for i, ro in enumerate(df_.iloc[1:-1, 4]):
    df_.iat[i+1, 4] = 'Tr' + str(i+1)

df_.iat[0, 4] = 'HOS'
df_.iat[-1, 4] = 'TN5'

df_.head(20)

,P,N,E,Z,D
0,1,9.879099e+06,278651.372306,1522.2,HOS
1,2,9.879092e+06,278786.621802,1524.6,Tr1
2,3,9.879104e+06,278831.631532,1525.0,Tr2
3,4,9.878829e+06,278862.250444,1523.6,Tr3
4,5,9.878798e+06,278778.551953,1521.0,Tr4
5,6,9.878723e+06,278761.448282,1520.7,Tr5
6,7,9.878684e+06,278823.967958,1521.3,Tr6
7,8,9.878603e+06,278853.595807,1521.5,Tr7
8,9,9.878613e+06,278934.641646,1522.4,Tr8
9,10,9.878682e+06,278935.016221,1522.5,Tr9


In [77]:
export = 'C:/Users/LUNA/Desktop/Traverse.csv'
df_.to_csv(export, index=False, header=False)
